In [1]:
from es import DocumentStore, Document

In [2]:
host = 'http://172.31.18.19:9200'

In [3]:
index = 'tmp_es_template'

In [4]:
storeclient = DocumentStore(host)

In [5]:
storeclient.create_store(index, lang='en', reset_index=True)

In [6]:
with open('sample.txt') as f:
    data = [Document(text=s.strip(), id=i) for i, s in enumerate(f.readlines())]

In [7]:
data

[Document(text='machine learning', id=0),
 Document(text='deep learning', id=1),
 Document(text='organizational behavior', id=2),
 Document(text='big disappointment', id=3)]

In [8]:
storeclient.add_documents(index=index, documents=data)

In [9]:
query = 'dissapoint'

In [10]:
storeclient.search_by_word_matching(index, query)

[]

In [11]:
storeclient.suggest(index=index, query_string=query)

([{'text': 'dissapoint', 'offset': 0, 'length': 10, 'options': []}],
 [{'text': 'dissapoint',
   'offset': 0,
   'length': 10,
   'options': [{'text': 'disappoint', 'score': 0.8, 'freq': 1}]}])